In [1]:
import sys
sys.path.append('C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO')
import argparse
import gc
import logging
import os
import sys
import time

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim

from traffino.data.loader import data_loader

In [2]:
# from traffino.losses import gan_g_loss, gan_d_loss, l2_loss
# from traffino.losses import displacement_error, final_displacement_error

In [2]:
from traffino.models import TrajectoryGenerator, TrajectoryDiscriminator
from traffino.utils import int_tuple, bool_flag, get_total_norm
from traffino.utils import relative_to_abs, get_dset_path

In [3]:
torch.backends.cudnn.benchmark = True # cudnn의 benchmark를 통해 최적 backend 연산을 찾는 flag를 true로 하겠음

In [4]:
parser = argparse.ArgumentParser()
FORMAT = '[%(levelname)s: %(filename)s: %(lineno)4d]: %(message)s'
logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [19]:
# args = parser.parse_args()
# # Dataset options
# parser.add_argument('--dataset_name', default='zara1', type=str)
# parser.add_argument('--delim', default=' ')
# parser.add_argument('--loader_num_workers', default=4, type=int)
# parser.add_argument('--obs_len', default=8, type=int)
# parser.add_argument('--pred_len', default=8, type=int)
# parser.add_argument('--skip', default=1, type=int)

# # Optimization
# parser.add_argument('--batch_size', default=64, type=int)
# parser.add_argument('--num_iterations', default=10000, type=int)
# parser.add_argument('--num_epochs', default=200, type=int)

# # Model Options
# parser.add_argument('--embedding_dim', default=64, type=int)
# parser.add_argument('--num_layers', default=1, type=int)
# parser.add_argument('--dropout', default=0, type=float)
# parser.add_argument('--batch_norm', default=0, type=bool_flag)
# parser.add_argument('--mlp_dim', default=1024, type=int)

# # Generator Options
# parser.add_argument('--encoder_h_dim_g', default=64, type=int)
# parser.add_argument('--decoder_h_dim_g', default=128, type=int)
# parser.add_argument('--noise_dim', default=None, type=int_tuple)
# parser.add_argument('--noise_type', default='gaussian')
# parser.add_argument('--noise_mix_type', default='ped')
# parser.add_argument('--clipping_threshold_g', default=0, type=float)
# parser.add_argument('--g_learning_rate', default=5e-4, type=float)
# parser.add_argument('--g_steps', default=1, type=int)

# # Pooling Options
# parser.add_argument('--pooling_type', default='pool_net')
# parser.add_argument('--pool_every_timestep', default=1, type=bool_flag)

# # Pool Net Option
# parser.add_argument('--bottleneck_dim', default=1024, type=int)

# # Social Pooling Options
# parser.add_argument('--neighborhood_size', default=2.0, type=float)
# parser.add_argument('--grid_size', default=8, type=int)

# # Discriminator Options
# parser.add_argument('--d_type', default='local', type=str)
# parser.add_argument('--encoder_h_dim_d', default=64, type=int)
# parser.add_argument('--d_learning_rate', default=5e-4, type=float)
# parser.add_argument('--d_steps', default=2, type=int)
# parser.add_argument('--clipping_threshold_d', default=0, type=float)

# # Loss Options
# parser.add_argument('--l2_loss_weight', default=0, type=float)
# parser.add_argument('--best_k', default=1, type=int)

# # Output
# parser.add_argument('--output_dir', default=os.getcwd())
# parser.add_argument('--print_every', default=5, type=int)
# parser.add_argument('--checkpoint_every', default=100, type=int)
# parser.add_argument('--checkpoint_name', default='checkpoint')
# parser.add_argument('--checkpoint_start_from', default=None)
# parser.add_argument('--restore_from_checkpoint', default=1, type=int)
# parser.add_argument('--num_samples_check', default=5000, type=int)

# # Misc
# parser.add_argument('--use_gpu', default=1, type=int)
# parser.add_argument('--timing', default=0, type=int)
# parser.add_argument('--gpu_num', default="0", type=str)

In [20]:
from typing import Tuple
class CreateArg():
    def __init__(self):
        # Dataset options
        self.dataset_name = 'waterloo'
        self.delim = '\t'
        self.loader_num_workers = 4
        self.obs_len = 8
        self.pred_len = 8
        self.skip = 1
        # Optimization
        self.batch_size = 64
        self.num_iterations = 10000
        self.num_epochs = 200                      
        # Model Options
        self.embedding_dim = 64
        self.num_layers = 1
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        self.neighborhood_size = 1024 # type=float
        self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        self.output_dir = os.getcwd()
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        self.checkpoint_name = 'checkpoint' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 1 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "0" # type=str   


args = CreateArg() 

In [6]:
print(args.use_gpu)

1


In [7]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight)


def get_dtypes(args):
    long_dtype = torch.LongTensor
    float_dtype = torch.FloatTensor
    if args.use_gpu == 1:
    # use_gpu == 1:
        long_dtype = torch.cuda.LongTensor
        float_dtype = torch.cuda.FloatTensor
    return long_dtype, float_dtype

In [23]:
long_dtype, float_dtype = get_dtypes(args)

In [8]:
# train_path = get_dset_path(args.dataset_name, 'train') # dset_name, dset_type
# print(train_path) # datasets\waterloo\train
train_path = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train'
val_path = get_dset_path(args.dataset_name, 'val')
print(val_path)

datasets\waterloo\val


In [9]:
logger.info("Initializing train dataset")
train_dset, train_loader = data_loader(args, train_path)
# logger.info("Initializing val dataset")
# _, val_loader = data_loader(args, val_path)

[INFO: 1364202733.py:    1]: Initializing train dataset
['0769_2.txt']


In [10]:
iterations_per_epoch = len(train_dset) / args.batch_size / args.d_steps
if args.num_epochs:
    args.num_iterations = int(iterations_per_epoch * args.num_epochs)

logger.info(
    'There are {} iterations per epoch'.format(iterations_per_epoch)
)

[INFO: 2378675245.py:    5]: There are 62.3359375 iterations per epoch


In [21]:
generator = TrajectoryGenerator(
    obs_len=args.obs_len,
    pred_len=args.pred_len,
    embedding_dim=args.embedding_dim,
    encoder_h_dim=args.encoder_h_dim_g,
    decoder_h_dim=args.decoder_h_dim_g,
    mlp_dim=args.mlp_dim,
    num_layers=args.num_layers,
    noise_dim=args.noise_dim,
    noise_type=args.noise_type,
    noise_mix_type=args.noise_mix_type,
    pooling_type=args.pooling_type,
    pool_every_timestep=args.pool_every_timestep,
    dropout=args.dropout,
    bottleneck_dim=args.bottleneck_dim,
    neighborhood_size=args.neighborhood_size,
    grid_size=args.grid_size,
    batch_norm=args.batch_norm)

In [25]:
generator.apply(init_weights)
generator.type(float_dtype).train()
logger.info('Here is the generator:')
logger.info(generator)

[INFO: 1295372059.py:    3]: Here is the generator:
[INFO: 1295372059.py:    4]: TrajectoryGenerator(
  (encoder): TrajEncoder(
    (encoder): LSTM(64, 64)
    (spatial_embedding): Linear(in_features=2, out_features=64, bias=True)
  )
  (decoder): Decoder(
    (decoder): LSTM(64, 128)
    (spatial_embedding): Linear(in_features=2, out_features=64, bias=True)
    (hidden2pos): Linear(in_features=128, out_features=2, bias=True)
    (pool_net): PoolHiddenNet(
      (spatial_embedding): Linear(in_features=2, out_features=64, bias=True)
      (mlp_pre_pool): Sequential(
        (0): Linear(in_features=192, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=1024, bias=True)
        (3): ReLU()
      )
    )
    (mlp): Sequential(
      (0): Linear(in_features=1152, out_features=1024, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=128, bias=True)
      (3): ReLU()
    )
  )
  (pool_net): PoolHiddenNet(
    (spatial

In [26]:
discriminator = TrajectoryDiscriminator(
    obs_len=args.obs_len,
    pred_len=args.pred_len,
    embedding_dim=args.embedding_dim,
    h_dim=args.encoder_h_dim_d,
    mlp_dim=args.mlp_dim,
    num_layers=args.num_layers,
    dropout=args.dropout,
    batch_norm=args.batch_norm,
    d_type=args.d_type)

In [27]:
discriminator.apply(init_weights)
discriminator.type(float_dtype).train()
logger.info('Here is the discriminator:')
logger.info(discriminator)

[INFO: 2576673517.py:    3]: Here is the discriminator:
[INFO: 2576673517.py:    4]: TrajectoryDiscriminator(
  (encoder): TrajEncoder(
    (encoder): LSTM(64, 64)
    (spatial_embedding): Linear(in_features=2, out_features=64, bias=True)
  )
  (real_classifier): Sequential(
    (0): Linear(in_features=64, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1, bias=True)
    (3): ReLU()
  )
)
